In [ ]:
!pip install shap
!pip install interpret
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Setting Up

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import plotly.express as px
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import StackingClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from imblearn.combine import SMOTETomek
pd.set_option('display.max_columns',30)
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from interpret.blackbox import LimeTabular
from interpret import show

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/PG32 CS3244/smoteNCTrain.csv')
df_test = pd.read_csv('/content/drive/MyDrive/PG32 CS3244/smoteNCTest.csv')

In [ ]:
df_train.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,0,0,0,0,135000.0,4,1,1,4,-13566,-1900,1,6,2,0
1,1,1,1,0,315000.0,4,3,3,4,-10328,-543,0,6,1,0
2,0,1,1,0,315000.0,0,1,2,4,-18184,-3021,0,8,1,0
3,0,0,1,1,180000.0,2,1,3,4,-13467,-3850,0,11,2,0
4,1,1,0,1,247500.0,4,1,1,4,-13086,-1931,0,8,3,0


In [ ]:
df_test.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_WORK_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,0,0,1,0,202500.0,2,1,2,4,-16834,-1692,0,11,1,0
1,1,1,0,0,675000.0,0,3,1,4,-18126,-948,0,10,2,0
2,0,0,0,0,234000.0,2,3,0,4,-21967,-5215,0,3,2,0
3,0,1,1,0,445500.0,0,3,1,4,-12477,-456,0,10,2,0
4,0,1,0,0,225000.0,4,1,1,5,-12155,-667,0,8,2,0


In [ ]:
df_train.shape

(50164, 15)

In [ ]:
df_test.shape

(21518, 15)

In [ ]:
X_train = df_train.iloc[:,1:-1] # all the variables
y_train = df_train.iloc[:,-1] # labels

X_test = df_test.iloc[:,1:-1] # all the variables
y_test = df_test.iloc[:,-1] # labels

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#### LIME for LightGBM Tuned

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
tuned_lgbm = LGBMClassifier(objective="binary",
  boosting_type= 'goss', 
  n_estimators= 570, 
  learning_rate= 0.56, 
  num_leaves= 480, 
  max_depth= 9, 
  min_data_in_leaf= 250, 
  max_bin= 280, 
  lambda_l1= 0, 
  lambda_l2= 45, 
  min_gain_to_split= 0.1, 
  bagging_fraction= 0.5, 
  feature_fraction= 0.5, 
  random_state=42
  )

In [ ]:
tuned_lgbm.fit(X_train, y_train)
y_predict = tuned_lgbm.predict(X_test)

In [ ]:
X_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [ ]:
lime = LimeTabular(predict_fn=tuned_lgbm.predict_proba, 
                   data=X_train, 
                   random_state=42)
# Get local explanations
# The index 10933 and 10943 are chosen because there's 5 instances of status 1 and 5 instances of status 0 inside.
lime_local = lime.explain_local(X_test[10933:10943], 
                                y_test[10933:10943], 
                                name='LIME')

In [ ]:
show(lime_local)

In [ ]:
lime1 = LimeTabular(predict_fn=tuned_lgbm.predict_proba, 
                   data=X_train, 
                   random_state=42)
# Get local explanations
lime1_local = lime1.explain_local(X_test[1585:1595], 
                                y_test[1585:1595], 
                                name='LIME')

In [ ]:
show(lime1_local)

#### LIME for RandomForest Tuned

In [ ]:
import random

from sklearn.ensemble import RandomForestClassifier
tuned_rf_model = RandomForestClassifier(random_state = 42,
  criterion= "gini", 
  n_estimators = 388, 
  max_depth= 18, 
  min_samples_split= 6, 
  min_samples_leaf= 1, 
  max_features= "auto")

In [ ]:
tuned_rf_model.fit(X_train, y_train)
y_predict = tuned_rf_model.predict(X_test)

In [ ]:
X_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [ ]:
lime = LimeTabular(predict_fn=tuned_rf_model.predict_proba, 
                   data=X_train, 
                   random_state=42)
# Get local explanations
lime_local = lime.explain_local(X_test[0:10], 
                                y_test[0:10], 
                                name='LIME')

In [ ]:
show(lime_local)

#### LIME for MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

tuned_MLP_model = MLPClassifier(hidden_layer_sizes=(10,50,100), max_iter=300,activation = 'relu',solver='adam',learning_rate='constant',alpha=0.05)

In [ ]:
tuned_MLP_model.fit(X_train, y_train)
y_predict = tuned_MLP_model.predict(X_test)

In [ ]:
X_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [ ]:
lime = LimeTabular(predict_fn=tuned_MLP_model.predict_proba, 
                   data=X_train, 
                   random_state=42)
# Get local explanations
lime_local = lime.explain_local(X_test[0:10], 
                                y_test[0:10], 
                                name='LIME')

In [ ]:
show(lime_local)

#### LIME for Catboost

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
tuned_catboost_model = CatBoostClassifier( random_state = 42, learning_rate = 0.1, iterations = 1000, loss_function = 'CrossEntropy')
tuned_catboost_model.fit(X_train, y_train, verbose=False)
y_predict = tuned_catboost_model.predict(X_test)

In [ ]:
X_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [ ]:
lime = LimeTabular(predict_fn=tuned_catboost_model.predict_proba, 
                   data=X_train, 
                   random_state=42)

lime_local = lime.explain_local(X_test[10933:10943], 
                                y_test[10933:10943], 
                                name='LIME')



In [ ]:
show(lime_local)

In [ ]:
lime = LimeTabular(predict_fn=tuned_catboost_model.predict_proba, 
                   data=X_train, 
                   random_state=42)

lime_local = lime.explain_local(X_test[10:20], 
                                y_test[10:20], 
                                name='LIME')


In [ ]:
show(lime_local)

In [ ]:
lime = LimeTabular(predict_fn=tuned_catboost_model.predict_proba, 
                   data=X_train, 
                   random_state=42)

lime_local = lime.explain_local(X_test[12670:12680], 
                                y_test[12670:12680], 
                                name='LIME')



In [ ]:
show(lime_local)